In [48]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder
import pickle

In [49]:
data=pd.read_csv("D:\Genai-lc-hf\Datasets\Churn_Modelling.csv")

In [50]:
# here i am predicting the estimated salary since it is a continuous value thus making this as a regression problem

# encode the categorical vars
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [51]:
#one-hot encoder 'Geography'
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [52]:
#combining
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [53]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,2,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,3,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,4,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,5,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [54]:
#splitting - Drop irrelevant columns that can't be scaled

# Drop RowNumber, CustomerId, Surname (non-numeric), EstimatedSalary (target), and Exited
X = data.drop(['RowNumber', 'CustomerId', 'Surname', 'EstimatedSalary', 'Exited'], axis=1)
y = data['EstimatedSalary']

In [55]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [56]:
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)


ANN regression Problem Statement 

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense


In [58]:
#model building
model=Sequential ([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #output layer for regression
    ])

#compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                768       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
#setup tensor board

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [61]:
#setup early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [62]:
#training

history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback]  
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 100394.2344 - mae: 100394.2344 - val_loss: 98574.3906 - val_mae: 98574.3906
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99804.7188 - mae: 99804.7188 - val_loss: 97347.9297 - val_mae: 97347.9297
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 97622.1719 - mae: 97622.1719 - val_loss: 94107.0156 - val_mae: 94107.0156
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 93230.4688 - mae: 93230.4688 - val_loss: 88591.7891 - val_mae: 88591.7891
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 86651.3047 - mae: 86651.3047 - val_loss: 81171.3594 - val_mae: 81171.3594
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 78542.3594 - mae: 78542.3594 - val_loss: 72854.8906 - val_mae: 72854.8906
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 70043.7500 - mae: 70043.7500 

In [63]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 4900), started 0:00:05 ago. (Use '!kill 4900' to kill it.)

In [67]:
#evaluate the model on the test data 

test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50404.2188 - mae: 50404.2188
Test MAE:50404.21875


In [68]:
model.save('regression_model.h5')

d:\Genai-lc-hf\tfenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
